# Item:
- propteries: [id, name, price]
- methods: getter + setter, toString, compareTo

# Order:
- properties: [id, account, item_list (dict), voucher (str)]
- methods: getVoucherPromo() - 10%, getBulkOrderPromo() - 10%, setScore(), printBill(), total()

# Account: 
- properties: [id, name, score]
- methods: getLoyaltyPromo() - 15%

In [1]:
class Item:
    def __init__(self, id: int, name: str, price: float) -> None:
        self.__id = id,
        self.__name = name
        self.__price = price
        
    # Get value
    @property
    def id(self) -> int:
        return self.__id
    @property
    def name(self) -> str:
        return self.__name
    @property
    def price(self) -> float:
        return self.__price
    
    # Set value
    @id.setter
    def id(self, id: int):
        self.__id = id
    @name.setter
    def name(self, name: str):
        self.__name = name
    @price.setter
    def price(self, price: float):
        self.__price = price
        
    def __str__ (self):
        return f"Item(id={self.__id}, name='{self.__name}', price={self.__price})"
    
    def __lt__(self, other):
        if isinstance(other, Item):
            return self.__id < other.__id
        return NotImplemented
    
    def compare_to(self, other):
        if not isinstance(other, Item):
            raise ValueError("Can only compare with another Item")
        return self.__id - other.__id

In [2]:
item1 = Item(1, "Milk", 2.49)
item2 = Item(2, "Bread", 1.99)
item3 = Item(3, "Eggs", 3.25)
item4 = Item(4, "Apples", 4.50)
item5 = Item(5, "Bananas", 2.99)
item6 = Item(6, "Cheese", 5.75)
item7 = Item(7, "Tomatoes", 3.10)
item8 = Item(8, "Chicken", 8.99)
item9 = Item(9, "Rice", 6.20)
item10 = Item(10, "Coffee", 7.80)

In [3]:
class Account:
    def __init__(self, id, name, score):
        pass

In [ ]:
class Order:
    def __init__(self, id:str, account:Account, item_list: dict = {}, voucher: str = ""):
        self.__id = id
        self.account = account
        self.__item_list = item_list
        self.__voucher = voucher
        
    # Get values
    @property
    def id(self):
        return self.__id
    @property
    def item_list(self):
        return self.__item_list  
    @property
    def voucher(self):
        return self.__voucher
    
    # Set values
    @item_list.setter
    def item_list(self, item_list:dict):
        self.__item_list = item_list
    @voucher.setter
    def voucher(self, voucher:str):
        self.__voucher = voucher
        
    # Add item to order
    def add_item(self, item: Item):
        if item.id not in self.__item_list:
            self.__item_list[item.id] = 1
        else:
            self.__item_list[item.id]+=1
            
    # Remove item from order
    def remove_item(self, item: Item):
        if item.id in self.__item_list:
            self.__item_list[item.id] -=1
            if self.__item_list[item.id] == 0:
                del self.__item_list[item.id]
                return True
            return True
        return False
    
    def getVoucherPromo(self):
        return 0.10 if self.voucher else 0.0
    
    def __getTotalQuantity(self):
        return sum(self.__item_list.values())
    
    def getBulkOrderPromo(self):
        return 0.10 if self.__getTotalQuantity() > 10 else 0.0
    
    def total(self):
        sum = 0
        for item in self.__item_list:
            qty = self.__item_list[item]
            sum += item.price * qty
        # tinh them giam gia
        sum *=  (1 - self.getBulkOrderPromo() - self.getVoucherPromo() - self.account.getLoyaltyPromo())
        return sum
    
    def setScore(self):
        return int(self.total() * 0.1)
    
    def printBill(self):
        with open(f"bill_{self.__id}.txt", "w") as file:
            file.write(f"Bill ID: {self.__id}\n")
            file.write(f"Voucher: {self.voucher}\n")
            file.write(f"Loyalty: {self.account.getLoyaltyPromo()}\n")
            file.write(f"Bulk Order: {self.getBulkOrderPromo()}\n")
            file.write(f"Total: {self.total()}\n")
            file.write(f"Score: {self.setScore()}\n")
            file.write("Items:\n")
            for item in self.__item_list:
                file.write(item.__str__())
                file.write(f"Quantity: {self.__item_list[item]}\n")
                file.write(f"Total: {self.total()}\n")